In [2]:
import pandas as pd
import numpy as np
rejected = pd.read_csv("Lending_Club_Rejected_2014_2018.csv")
accepted = pd.read_csv("Lending_Club_Accepted_2014_2018.csv")

/Users/sunying/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
libor_14_to_18 = pd.read_csv("LIBOR14to18.csv")

In [49]:
def modify_date(input_date): #Oct-2014 >>> 201410 in str
    input_year = input_date[-4:]
    input_month = input_date[0:3]
    month_dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", 
    "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}
    month = month_dict[input_month]
    return input_year + month

trimmed = accepted[["int_rate", "issue_d", "term"]]
def trim_func(str):
    return str[0:2]
trimmed["int_index"] = trimmed['int_rate']*(trimmed['term'].apply(trim_func).astype("float"))
trimmed_twice = trimmed[["int_index", "issue_d"]]
trimmed_twice.issue_d = trimmed_twice["issue_d"].apply(modify_date)
trimmed_twice.sort_values(by=['issue_d'])
int_index_monthly = trimmed_twice.groupby(by=['issue_d']).mean()

/var/folders/3x/vz5rzs_16ss0fjxrltm5qnv00000gn/T/ipykernel_66573/838185056.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["int_index"] = trimmed['int_rate']*(trimmed['term'].apply(trim_func).astype("float"))
/Users/sunying/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [55]:
# from matplotlib import pyplot as plt
x = libor_14_to_18[libor_14_to_18.columns[1]].to_numpy()
y = int_index_monthly[int_index_monthly.columns[0]].to_numpy()
# plt.xlabel("libor") 
# plt.ylabel("int_index") 
# plt.plot(x,y)
corr = np.corrcoef(x, y)
print(corr[0,1])

-0.3668372525653875


In [19]:
def trim_accept(accept):

    cols = ['loan_amnt', 'title','dti','zip_code','addr_state','emp_length','policy_code']
    trimmed_acp = accept[cols]
    return trimmed_acp 

def trim_rej(rej):
    return rej[["Amount_Requested", "Loan_Title", "Debt_to_Income_Ratio", "Zip_Code", "State", "Employment_Length", "Policy_Code"]]

rejected_dropna = pd.DataFrame(trim_rej(rejected).dropna())
accepted_dropna = pd.DataFrame(trim_accept(accepted).dropna())
accepted_dropna.columns = ['Amount_Requested', 'Loan_Title', 'Debt_to_Income_Ratio', 'Zip_Code', 'State', 'Employment_Length', 'Policy_Code']
accepted_dropna["Status"] = 1
rejected_dropna["Status"] = 0
rejected_dropna["Debt_to_Income_Ratio"] = rejected_dropna["Debt_to_Income_Ratio"].str.rstrip("%").astype("float") 
frames = [accepted_dropna,rejected_dropna]
merged = pd.concat(frames)
merged.Employment_Length = merged.Employment_Length.str[:2]
merged.loc[ merged["Employment_Length"] == "< ", "Employment_Length"] = 0.1
merged["Employment_Length"] = merged["Employment_Length"].astype("float") 

In [17]:
print(len(merged["State"].to_numpy()[0]))

2


In [38]:
income_by_state = pd.read_csv("state_average_income_2016.csv")
states = income_by_state[income_by_state.columns[0]].astype("str").to_numpy()[0:51]
# income_by_state["income"] = income_by_state["income"].str.rstrip(",")
# print(income_by_state.columns["income"])
incomes = income_by_state[income_by_state.columns[1]].to_numpy()[0:51]

income_by_state_dict = dict(zip(states, incomes))
print(income_by_state_dict)
def modify_state(state):
    income_by_state_dict[str(state)]
print(modify_state("PA"))
# merged.State = merged["State"].apply(modify_state)
# print(merged.shape)

{'AL': '50,936', 'AK': '81,680', 'AZ': '61,592', 'AR': '49,518', 'CA': '71,879', 'CO': '76,117', 'CT': '81,895', 'DE': '62,612', 'DC': '76,566', 'FL': '55,202', 'GA': '57,738', 'HI': '77,807', 'ID': '61,014', 'IL': '66,215', 'IN': '60,507', 'IA': '63,743', 'KS': '61,279', 'KY': '48,938', 'LA': '45,515', 'ME': '54,857', 'MD': '79,562', 'MA': '77,951', 'MI': '61,582', 'MN': '75,742', 'MS': '44,332', 'MO': '59,344', 'MT': '61,565', 'NE': '64,045', 'NV': '59,791', 'NH': '82,259', 'NJ': '73,854', 'NM': '52,262', 'NY': '66,270', 'NC': '57,993', 'ND': '64,918', 'OH': '58,232', 'OK': '54,950', 'OR': '63,787', 'PA': '65,776', 'RI': '66,368', 'SC': '58,610', 'SD': '61,969', 'TN': '55,383', 'TX': '62,720', 'UT': '72,789', 'VT': '65,623', 'VA': '71,678', 'WA': '75,841', 'WV': '47,843', 'WI': '64,522', 'WY': '62,378'}
None


In [ ]:
merged = merged.sample(frac=1).reset_index(drop=True)